In [83]:
###
#Goal: See how varying SRM curvature will effect the mode-matching in OMC
#First we use all of the aLIGO cavities to derive the ideal mode-matching parameters
#then we extract the beam quality from BS to SRC
#
# then we turn off all the cavities and inject the beam.
###

In [84]:
%matplotlib inline
import pykat
import pylab as pl
import pandas
import matplotlib.pyplot as plt
import matplotlib
import csv
import math
import numpy as np
from pykat.parallel import parakat

pandas.set_eng_float_format(accuracy=3, use_eng_prefix=True)

In [85]:
####Perfect Mode-Matching Version:
katfile = "aLIGO_IFO_AWC_MM.kat"

basekat = pykat.finesse.kat()
basekat.loadKatFile(katfile)

# Add cavity parameter detectors
basekat.parseCommands("""

ad inputa 0 nSRBS

bp inputxr x r nSRBS
bp inputyr y r nSRBS

bp inputxw x w nSRBS
bp inputyw y w nSRBS

bp inputxq x q nSRBS
bp inputyq y q nSRBS

cp cavXARM x w
cp cavXARM x w0
cp cavXARM x z
cp cavXARM x r
cp cavXARM x q
cp cavXARM x length
cp cavXARM x loss

cp cavYARM x w
cp cavYARM x w0
cp cavYARM x z
cp cavYARM x r
cp cavYARM x q
cp cavYARM x length
cp cavYARM x loss

cp cavOMC x w
cp cavOMC x w0
cp cavOMC x z
cp cavOMC x r
cp cavOMC x q
cp cavOMC x length
cp cavOMC x loss

bp outputxq x q nOMC_HROC_refl
bp outputyq y q nOMC_HROC_refl

yaxis lin re:im

""")


#kat.cavXARM.enabled=True
#kat.cavYARM.enabled=True

basekat.undumpNodes() #undump the vacuum nodes
kat=basekat.deepcopy() #copy kat code
kat.noxaxis = True
kat.verbose = False

for cav in kat.getAll(pykat.commands.cavity):
    cav.enabled=True

#kat.cavOMC.enabled=False

out  = kat.run(getTraceData=True)

Parsing `put lx1 L $Lmx` into pykat object not implemented yet, added as extra line.
Parsing `put ly1 L $Lmy` into pykat object not implemented yet, added as extra line.


In [86]:
###Extract the beam parameters at BS->SRM
inputxw = float(np.real(out[0]['inputxw']))
inputxr = float(np.real(out[0]['inputxr']))
inputxq = out[0]['inputxq']


inputyw = float(np.real(out[0]['inputyw']))
inputyr = float(np.real(out[0]['inputyr']))
inputyq = out[0]['inputyq']

print(inputxw, inputxr, inputxq)

c = 3.0e8
n = 1.0
eps = 8.854e-12

input_I = (float(np.absolute(out[0]['inputa'])**2)) * (c * n * eps/2 )           
avg_w = (inputxw +inputyw)/2
input_P = input_I * 4 *np.pi * avg_w**2

#look at nodal trace of full IFO
print(out[1][0]['nSRBS'][0])
print(out[1][0]['nSRBS'][1])

0.0531929289458351 -1392.84977207358 (-1355.18165076+225.936156534j)
(-1355.1893788291+225.9582195075j)
(-1362.46207183065+203.666481904717j)


In [87]:
#### Just to idiot check, here are the perfect mode-matching parameteres of OMC (Eigenmodes)
omc_z_1 = np.real(out[0]['cavOMC_x_z'])
print(omc_z_1)

omc_q_1 = out[0]['cavOMC_x_q']
print(omc_q_1)

omc_r_1 = np.real(out[0]['cavOMC_x_r'])
print(omc_r_1)

omc_w_1 = np.real(out[0]['cavOMC_x_w'])
print(omc_w_1)

omc_l_1 = np.real(out[0]['cavOMC_x_length'])
print(omc_l_1)

omc_l_1 = np.real(out[0]['cavOMC_x_loss'])
print(omc_l_1)

# doublecheck q values
lam = 1064e-9
pi = 3.1415926
q_check =1/( (1/omc_r_1) - 1j * (lam/(pi*omc_w_1**2)) )
print(q_check)

0.140764742971947
(0.140764742972+0.707540463523j)
3.69714894084054
0.000499115038950642
1.1314
0.0151533625445898
(0.140764738352+0.707540452373j)


In [88]:
katfile1 = "aLIGO_IFO_AWC_MM_slim.kat"

basekat = pykat.finesse.kat()
basekat.loadKatFile(katfile1)
basekat.undumpNodes() #undump the vacuum nodes

#gauss_string = 'gauss** gaussin L0 nSRBS ' + str(inputxw) + ' ' \
#+ str(inputxr)+' '\
#+ str(inputyw) + ' '\
#+ str(inputyr)

gauss_string = 'gauss* gaussin L0 nSRBS ' + str(float(np.real(inputxq))) +' '+ str(float(np.imag(inputxq)))+ ' '\
 + str(float(np.real(inputyq))) +' '+ str(float(np.imag(inputyq)))
print(gauss_string)
basekat.parseCommands(gauss_string)
basekat.parseCommands("""
startnode nSRBS

bp outputxq x q nOMC_HROC_in
bp outputyq y q nOMC_HROC_in

bp outputxz x z nOMC_HROC_in
bp outputyz y z nOMC_HROC_in

yaxis lin re:im
""")

gauss* gaussin L0 nSRBS -1355.18165075501 225.936156533966 -1362.45449772394 203.641231560045
Parsing `startnode nSRBS` into pykat object not implemented yet, added as extra line.


In [89]:
kat1=basekat.deepcopy() #copy kat code
kat1.noxaxis = True
kat1.verbose = False

for cav in kat1.getAll(pykat.commands.cavity):
    cav.enabled=False

#kat1.cavOMC.enabled=True

out1  = kat1.run(getTraceData=True)

In [90]:
kat2=basekat.deepcopy() #copy kat code
kat2.noxaxis = True
kat2.verbose = False

for cav in kat2.getAll(pykat.commands.cavity):
    cav.enabled=False

#kat2.cavOMC.enabled=True

kat2.SRMHR.Rc = 'inf'
out2 = kat2.run(getTraceData=True)

In [91]:
kat3=basekat.deepcopy() #copy kat code
kat3.noxaxis = True
kat3.verbose = False

for cav in kat3.getAll(pykat.commands.cavity):
    cav.enabled=False

#kat3.cavOMC.enabled=True

kat3.SRMHR.Rc = -6
out3 = kat3.run(getTraceData=True)

In [92]:
print(out[1][0]['nSRBS'][0])
print(out1[1][0]['nSRBS'][0])
print(out2[1][0]['nSRBS'][0])
print(out3[1][0]['nSRBS'][0])

print(out[1][0]['nOMC_HROC_in'][0])
print(out1[1][0]['nOMC_HROC_in'][0])
print(out2[1][0]['nOMC_HROC_in'][0])
print(out3[1][0]['nOMC_HROC_in'][0])

(-1355.1893788291+225.9582195075j)
(-1355.18165075501+225.936156533966j)
(-1355.18165075501+225.936156533966j)
(-1355.18165075501+225.936156533966j)
(0.140764742971948+0.707540463523089j)
(0.623844951721355+0.00711441803645835j)
(0.574020623680087+0.0043270094126464j)
(0.620632073154188+0.00691382203221927j)


In [93]:
print(out1[0]['outputxz'])
print(out2[0]['outputxz'])
print(out3[0]['outputxz'])

print(omc_z_1)

(0.623844951721+0j)
(0.57402062368+0j)
(0.620632073154+0j)
0.140764742971947


In [94]:
vars(kat1.SRMHR.Rc)

{'_canFsig': False,
 '_component_name': 'SRMHR',
 '_isPutable': True,
 '_isPutter': True,
 '_isTunable': True,
 '_name': 'Rcx',
 '_owner': <weakref at 0x7f0dfc18aae8; to 'pykat.components.mirror_3103' at 0x7f0dfc369048>,
 '_parameter_name': 'Rcx',
 '_put_name': 'var_SRMHR_Rcx',
 '_putter': None,
 '_value': -5.6938,
 'put_count': 0,
 'putees': []}

In [95]:
vars(kat2.SRMHR.Rc)

{'_canFsig': False,
 '_component_name': 'SRMHR',
 '_isPutable': True,
 '_isPutter': True,
 '_isTunable': True,
 '_name': 'Rcx',
 '_owner': <weakref at 0x7f0dfc18aae8; to 'pykat.components.mirror_3103' at 0x7f0dfc369048>,
 '_parameter_name': 'Rcx',
 '_put_name': 'var_SRMHR_Rcx',
 '_putter': None,
 '_value': inf,
 'put_count': 0,
 'putees': []}

In [96]:
vars(kat3.SRMHR.Rc)

{'_canFsig': False,
 '_component_name': 'SRMHR',
 '_isPutable': True,
 '_isPutter': True,
 '_isTunable': True,
 '_name': 'Rcx',
 '_owner': <weakref at 0x7f0dfc18aae8; to 'pykat.components.mirror_3103' at 0x7f0dfc369048>,
 '_parameter_name': 'Rcx',
 '_put_name': 'var_SRMHR_Rcx',
 '_putter': None,
 '_value': -6.0,
 'put_count': 0,
 'putees': []}

In [97]:
vars(out[0])

{'StartDateTime': datetime.datetime(2016, 6, 13, 9, 28, 53, 112611),
 'katScript': '% Generated by PyKat 13.06.2016 09:28:53\n\n%%% FTblock Laser\nl L0 125.0 0.0 0.0 n0\ns lmod1 1.0 n0 n1\nmod mod1 9099471.0 0.18 1 pm 0.0 n1 n2\ns lmod2 1.0 n2 n3\nmod mod2 45497355.0 0.18 1 pm 0.0 n3 n4\ns lmod3 1.0 n4 n5\nmod mod3 24000000.0 0.1 2 pm 0.0 n5 n6\ns mod3toMC1 1.0 n6 nMC1in\n%%% FTend Laser\n\n%%% FTblock IMC\nbs MC1 0.9939694 0.00603 0.0 44.59 nMC1in nMC1refl nMC1trans nMC1fromMC3\ns sMC1toMC2 16.2405708 nMC1trans nMC2in\nbs MC2 0.9999856 5.1e-06 0.0 0.82 nMC2in nMC2refl nMC2trans dump_0\nattr MC2 Rcx 27.275\nattr MC2 Rcy 27.275\ns sMC2toMC3 16.2405708 nMC2refl nMC3in\nbs MC3 0.9941542 0.005845 0.0 44.59 nMC3in nMC3refl nMC3trans nMCreturn_refl\ns sMC3toMC1 0.465 nMC3refl nMC1fromMC3\ns sMC3substrate 0.0845 1.44963098985906 nMC3trans nMC3ARin\nbs MC3AR 0.0 1.0 0.0 28.9661 nMC3ARin dump_3 nMC3ARtrans dump_4\ns sMC3ARtoIM1 0.4282 nMC3ARtrans nIM1in\n%%% FTend IMC\n\n%%% FTblock HAM2\nbs IM